# Run simulation with interactive OMC session
This example illustrates how to develop, run, test(, repeat) a Modelica model in a (local) pandas environment, enabling quick iterations to analyze and tune the model before going to Big Data.

See https://openmodelica.org/doc/OpenModelicaUsersGuide/v1.11.0/ompython.html

In [ ]:
import pandas as  pd
import os
import sys

In [ ]:
# Finds the package in the repo instead of the installed one
sys.path.insert(0, '../../../src')

In [ ]:
import tempfile
from OMPython import ModelicaSystem
from ModelicaModels import EDrives

In [ ]:
# mod = BouncingBall.instantiatemodel()  # Does NOT connect to omc
modelname = EDrives.modelName
# xmlFilePath = os.path.join(SETUP_DIR, 'ModelicaModels', 'build', modelname)
mod = ModelicaSystem(
        fileName=os.path.dirname(EDrives.__file__) + '/' + modelname.replace('.', '/')+".mo",
        modelName=modelname, lmodel='Modelica',
    )

In [ ]:
sim_options_d = mod.getSimulationOptions()
sim_options_d['stopTime'] = 2
mod.setSimulationOptions(sim_options_d)

In [ ]:
mod.getSimulationOptions()

In [ ]:
%%time
resfilename = os.path.join(tempfile.gettempdir(), 'b.mat')
mod.setParameters("e=0.2")
mod.simulate(resultfile=resfilename, simflags=None)

In [ ]:
# Needs connection to omc
vars = mod.getSolutions()
vars

In [ ]:
%%time
# TODO Loading results (from mat-file) is very slow
data = mod.getSolutions(varList=list(vars), resultfile=resfilename)
ts_df = pd.DataFrame(data).T
ts_df.columns = vars
ts_df

In [ ]:
%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.5))
ax.plot(ts_df['time'], ts_df['h'])
plt.tight_layout()